In [1]:
import numpy as np
import networkx as nx
import time
import random
from copy import deepcopy as dc

In [3]:
def generate_random_graph(degre_sequance):
    G = nx.configuration_model(degre_sequance,create_using=None, seed=None)
    G = nx.Graph(G)
    G.remove_edges_from(G.selfloop_edges())

    num_of_edges = nx.number_of_edges(G)
    edges_list = G.edges()

    if num_of_edges > 30000:
        edges_to_drop = num_of_edges - 30000
        x = np.random.choice(num_of_edges,edges_to_drop,replace=False)
        for i in x:
            a,b = edges_list[i]
            G.remove_edge(a,b)
    elif num_of_edges < 30000:
        edges_to_add = 30000 - num_of_edges
        x = np.random.choice(10000,edges_to_add*2,replace=False)
        to_add_list = zip(x[:edges_to_add],x[edges_to_add:])
        G.add_edges_from(to_add_list)
    else:
        pass
    
    return G

In [2]:
class Diffuse: #默认网络结构为节点数量为10000，边为60000的单向随机网络
    repetes = 10 #默认多次扩散重复次数
    def __init__(self,p,q,num_runs,G=nx.gnm_random_graph(10000,30000)):
        self.p = p
        self.q = q
        self.G = G.to_directed()
        self.num_runs = num_runs
        
    def decision(self,i): #个体状态决策规则
        influ = len([k for k in self.DG[i].get('prede',[]) if self.DG[k]['state'] == 1])           
        prob = self.p + self.q*influ
        if random.random() <= prob:
            return True
        else:
            return False
        
    def single_diffuse(self): #单次扩散
        self.DG = dc(self.G) #取网络的深层copy，以使原网络不被“污染” 
        for i in self.DG.nodes():
            self.DG[i]['prede'] = self.DG.predecessors(i)
            self.DG[i]['state'] = 0
        non_adopt_set = [i for i in self.DG.nodes() if self.DG[i]['state'] == 0]
        num_of_adopt = []
        j = 1
        while j <= self.num_runs:                                 
            x = 0
            random.shuffle(non_adopt_set)
            for i in non_adopt_set:                   
                if self.decision(i):
                    self.DG[i]['state'] = 1
                    non_adopt_set.remove(i)
                    x = x+1
            num_of_adopt.append(x)
            j = j+1
        return num_of_adopt
    
    def repete_diffuse(self): #多次扩散
        adopt_cont = []
        for i in range(self.repetes):
            num_of_adopt = self.single_diffuse()
            adopt_cont.append(num_of_adopt)
        return adopt_cont

In [8]:
text_set = ['ws0','ws0.1','ws0.3','ws0.5','ws0.7','ws0.9']

In [9]:
n = 10000
k = 3
nwk_cont = [nx.watts_strogatz_graph(10000,6,p=0),nx.watts_strogatz_graph(10000,6,p=0.1),nx.watts_strogatz_graph(10000,6,p=0.3),
                         nx.watts_strogatz_graph(10000,6,p=0.5),nx.watts_strogatz_graph(10000,6,p=0.7),nx.watts_strogatz_graph(10000,6,p=0.9)]

In [10]:
pq_range = [(i,j) for i in np.arange(0.0005,0.0255,0.001) for j in np.arange(0.02,0.2,0.003)] #25*60=1500

In [11]:
for u in range(6):
    G = nwk_cont[u]
    time1 = time.clock()
    print '------------------%s------------------'%text_set[u]
    diff_cont = []
    k = 1
    for p,q in pq_range:
        diff = Diffuse(p,q,25,G)
        temp = diff.repete_diffuse()
        x = np.mean(temp,axis=0)
        x = np.insert(x,0,[p,q])
        diff_cont.append(x)
        k = k+1
        if k in [100,400,800,1200,1400]:
            print '%s'%text_set[u],k,'runs:',time.clock()-time1,'s'
            time1 = time.clock()

    to_write = np.array(diff_cont)
    np.save("diffusion data set for %s"%text_set[u],to_write)

------------------ws0------------------
ws0 100 runs: 1383.4213828 s
ws0 400 runs: 4077.89347918 s
ws0 800 runs: 5070.15732564 s
ws0 1200 runs: 4802.10860721 s
ws0 1400 runs: 2311.33291556 s
------------------ws0.1------------------
ws0.1 100 runs: 1538.47387942 s
ws0.1 400 runs: 3991.52070191 s
ws0.1 800 runs: 4929.06047409 s
ws0.1 1200 runs: 4678.10498725 s
ws0.1 1400 runs: 2267.91892179 s
------------------ws0.3------------------
ws0.3 100 runs: 1356.43021102 s
ws0.3 400 runs: 3789.88494974 s
ws0.3 800 runs: 4778.19494814 s
ws0.3 1200 runs: 4620.96230793 s
ws0.3 1400 runs: 2246.25365574 s
------------------ws0.5------------------
ws0.5 100 runs: 1375.17276569 s
ws0.5 400 runs: 3767.88188945 s
ws0.5 800 runs: 4750.23177093 s
ws0.5 1200 runs: 4609.2115631 s
ws0.5 1400 runs: 2243.50921083 s
------------------ws0.7------------------
ws0.7 100 runs: 1342.79173298 s
ws0.7 400 runs: 3756.28759299 s
ws0.7 800 runs: 4749.53405985 s
ws0.7 1200 runs: 4613.79018287 s
ws0.7 1400 runs: 2246.36635

In [ ]:
text_set = ['er','ba','exp','gau','lognorm','ws0','ws0.1','ws0.3','ws0.5','ws0.7','ws0.9','sigma-0.1','sigma-0.2','sigma-0.4','sigma-0.6','sigma-0.8','sigma-1','sigma-1.2']